# ABE 516 Project Data Wrangling 2

## Nitrate Concentration Data

In [1]:
#imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime as dt

# allow plots to appear directly in the notebook
%matplotlib inline

**Read in Data**

In [35]:
wq_raw = pd.read_csv("SH_WQ_raw_2022.csv")
wq_raw.head()

,Case Number,Scientist,Matrix,Project Name,Submission Date,Comments,Units,Sample Number,Site ID,Bottle No,NO3-N
0,22x0132,Natalia Rogovska,Water,Shearer,4/12/2022,NaN,mg/L,22x0132-001,SH210101,1.0,3.3
1,22x0132,Natalia Rogovska,Water,Shearer,4/12/2022,NaN,mg/L,22x0132-002,SH210102,2.0,0.4
2,22x0132,Natalia Rogovska,Water,Shearer,4/12/2022,NaN,mg/L,22x0132-003,SH210103,3.0,< 0.3
3,22x0132,Natalia Rogovska,Water,Shearer,4/12/2022,NaN,mg/L,22x0132-004,SH210104,4.0,6.1
4,22x0132,Natalia Rogovska,Water,Shearer,4/12/2022,NaN,mg/L,22x0132-005,SH210105,5.0,5.8


**Change Data Types**

In [36]:
wq_raw.dtypes

Case Number         object
Scientist           object
Matrix              object
Project Name        object
Submission Date     object
Comments            object
Units               object
Sample Number       object
Site ID             object
Bottle No          float64
NO3-N               object
dtype: object

In [37]:
wq_raw = wq_raw.astype({"Case Number":"string",
                       "Scientist":"string",
                       "Matrix":"string",
                       "Project Name":"string", 
                       "Comments":"string",
                       "Units":"string",
                       "Sample Number":"string",
                       "Site ID":"string", 
                        })
wq_raw["Submission Date"]=pd.to_datetime(wq_raw["Submission Date"])
print(wq_raw.dtypes)

Case Number                string
Scientist                  string
Matrix                     string
Project Name               string
Submission Date    datetime64[ns]
Comments                   string
Units                      string
Sample Number              string
Site ID                    string
Bottle No                 float64
NO3-N                      object
dtype: object


**Change Blanks so conversion of datatypes later is correct**

In [38]:
wq_raw.loc[wq_raw["Site ID"] == "BLANK", "Site ID"] = "000000BL"

### Separate Site ID Column into Separate Columns

In [39]:
wq_raw["Site ID"][0][0:1]

'S'

In [44]:
wq_raw["Site Code"]= wq_raw["Site ID"].str.slice(0,2)
wq_raw["Year Code"]= wq_raw["Site ID"].str.slice(2,3)
wq_raw["DOY"]= wq_raw["Site ID"].str.slice(3,6)
wq_raw["Sample Location"]= wq_raw["Site ID"].str.slice(6,8)
wq_raw.head()
wq_raw[5:25]

,Case Number,Scientist,Matrix,Project Name,Submission Date,Comments,Units,Sample Number,Site ID,Bottle No,NO3-N,Site Code,Year Code,DOY,Sample Location
5,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-006,SH210106,6.0,1.9,SH,2,101,06
6,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-007,SH2101BX,7.0,11.2,SH,2,101,BX
7,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-008,SH2101CK,8.0,13.6,SH,2,101,CK
8,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-009,000000BL,9.0,< 0.3,00,0,000,BL
9,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-001,SH211601,1.0,2.2,SH,2,116,01
10,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-002,SH211602,2.0,0.6,SH,2,116,02
11,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-003,SH211603,3.0,< 0.3,SH,2,116,03
12,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-004,SH211604,4.0,5.9,SH,2,116,04
13,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-005,SH211605,5.0,6.6,SH,2,116,05
14,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-006,SH211606,6.0,4,SH,2,116,06


In [41]:
wq_raw.dtypes

Case Number                string
Scientist                  string
Matrix                     string
Project Name               string
Submission Date    datetime64[ns]
Comments                   string
Units                      string
Sample Number              string
Site ID                    string
Bottle No                 float64
NO3-N                      object
Site Code                  string
Year Code                  string
DOY                        string
Sample Location            string
dtype: object

In [46]:
wq_raw["DOY"] = wq_raw["DOY"].astype(int)

,Case Number,Scientist,Matrix,Project Name,Submission Date,Comments,Units,Sample Number,Site ID,Bottle No,NO3-N,Site Code,Year Code,DOY,Sample Location
5,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-006,SH210106,6.0,1.9,SH,2,101,06
6,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-007,SH2101BX,7.0,11.2,SH,2,101,BX
7,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-008,SH2101CK,8.0,13.6,SH,2,101,CK
8,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-009,000000BL,9.0,< 0.3,00,0,0,BL
9,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-001,SH211601,1.0,2.2,SH,2,116,01
10,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-002,SH211602,2.0,0.6,SH,2,116,02
11,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-003,SH211603,3.0,< 0.3,SH,2,116,03
12,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-004,SH211604,4.0,5.9,SH,2,116,04
13,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-005,SH211605,5.0,6.6,SH,2,116,05
14,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-006,SH211606,6.0,4,SH,2,116,06


**Create Sample Date Column**

In [55]:
MMDD = "01/01"
YY = wq_raw["Year Code"][0]+wq_raw["Year Code"][1] # Use the first and second rows as a check
first_day_of_year = MMDD+"/"+YY
print(first_day_of_year)

01/01/22


In [58]:
first_day_of_year = pd.to_datetime(first_day_of_year)

In [70]:
wq_raw["Sample Date"] = first_day_of_year
wq_raw["Sample Date"] = wq_raw["Sample Date"] + pd.to_timedelta((wq_raw["DOY"]-1), unit="d")
#first_day_of_year + dt.timedelta(days = wq_raw["DOY"])
wq_raw.head()

,Case Number,Scientist,Matrix,Project Name,Submission Date,Comments,Units,Sample Number,Site ID,Bottle No,NO3-N,Site Code,Year Code,DOY,Sample Location,Sample Date
0,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-001,SH210101,1.0,3.3,SH,2,101,01,2022-04-11
1,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-002,SH210102,2.0,0.4,SH,2,101,02,2022-04-11
2,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-003,SH210103,3.0,< 0.3,SH,2,101,03,2022-04-11
3,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-004,SH210104,4.0,6.1,SH,2,101,04,2022-04-11
4,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-005,SH210105,5.0,5.8,SH,2,101,05,2022-04-11


In [71]:
wq_raw[5:25]

,Case Number,Scientist,Matrix,Project Name,Submission Date,Comments,Units,Sample Number,Site ID,Bottle No,NO3-N,Site Code,Year Code,DOY,Sample Location,Sample Date
5,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-006,SH210106,6.0,1.9,SH,2,101,06,2022-04-11
6,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-007,SH2101BX,7.0,11.2,SH,2,101,BX,2022-04-11
7,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-008,SH2101CK,8.0,13.6,SH,2,101,CK,2022-04-11
8,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-009,000000BL,9.0,< 0.3,00,0,0,BL,2021-12-31
9,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-001,SH211601,1.0,2.2,SH,2,116,01,2022-04-26
10,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-002,SH211602,2.0,0.6,SH,2,116,02,2022-04-26
11,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-003,SH211603,3.0,< 0.3,SH,2,116,03,2022-04-26
12,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-004,SH211604,4.0,5.9,SH,2,116,04,2022-04-26
13,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-005,SH211605,5.0,6.6,SH,2,116,05,2022-04-26
14,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-006,SH211606,6.0,4,SH,2,116,06,2022-04-26


**Replace Nitrate Concentration values below detection limit with zero**

In [75]:
wq_raw["NO3-N"] = wq_raw["NO3-N"].astype(str)

In [81]:
wq_raw["NO3-N-clean"] = np.where(wq_raw["NO3-N"].str.contains("<"), "0.0", wq_raw["NO3-N"])
wq_raw["NO3-N-clean"] = wq_raw["NO3-N-clean"].astype(float)
wq_raw[5:15]

,Case Number,Scientist,Matrix,Project Name,Submission Date,Comments,Units,Sample Number,Site ID,Bottle No,NO3-N,Site Code,Year Code,DOY,Sample Location,Sample Date,NO3-N-clean
5,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-006,SH210106,6.0,1.9,SH,2,101,06,2022-04-11,1.9
6,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-007,SH2101BX,7.0,11.2,SH,2,101,BX,2022-04-11,11.2
7,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-008,SH2101CK,8.0,13.6,SH,2,101,CK,2022-04-11,13.6
8,22x0132,Natalia Rogovska,Water,Shearer,2022-04-12,<NA>,mg/L,22x0132-009,000000BL,9.0,< 0.3,00,0,0,BL,2021-12-31,0.0
9,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-001,SH211601,1.0,2.2,SH,2,116,01,2022-04-26,2.2
10,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-002,SH211602,2.0,0.6,SH,2,116,02,2022-04-26,0.6
11,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-003,SH211603,3.0,< 0.3,SH,2,116,03,2022-04-26,0.0
12,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-004,SH211604,4.0,5.9,SH,2,116,04,2022-04-26,5.9
13,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-005,SH211605,5.0,6.6,SH,2,116,05,2022-04-26,6.6
14,22x0167,Natalia Rogovska,Water,Shearer,2022-04-28,<NA>,mg/L,22x0167-006,SH211606,6.0,4,SH,2,116,06,2022-04-26,4.0


### Create Final Clean NO3-N Concentration Dataset

In [82]:
wq_clean = wq_raw[["Sample Date", "Site ID", "Site Code", "Sample Location", "NO3-N-clean"]]
wq_clean

,Sample Date,Site ID,Site Code,Sample Location,NO3-N-clean
0,2022-04-11,SH210101,SH,01,3.3000
1,2022-04-11,SH210102,SH,02,0.4000
2,2022-04-11,SH210103,SH,03,0.0000
3,2022-04-11,SH210104,SH,04,6.1000
4,2022-04-11,SH210105,SH,05,5.8000
...,...,...,...,...,...
103,2022-08-05,SH221704,SH,04,15.2390
104,2022-08-05,SH221705,SH,05,16.0490
105,2022-08-05,SH221706,SH,06,9.5212
106,2022-08-05,SH2217BX,SH,BX,19.4780


In [84]:
wq_clean.to_csv("no3_conc.csv", index = False)